In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import json
import heapq
from google.colab.patches import cv2_imshow

In [ ]:
%cd /content/drive/MyDrive/Colab_Notebooks/CV/CoursePro-2/

/content/drive/MyDrive/Colab_Notebooks/CV/CoursePro-2


In [ ]:
root = '/content/drive/MyDrive/Colab_Notebooks/CV/CoursePro-2/'

In [ ]:
image_folder = '/content/drive/MyDrive/Colab_Notebooks/CV/CoursePro-2/images'

In [ ]:
with open(os.path.join(root, "groundtruth.json"), 'r') as file:
    json_data = file.read()

In [ ]:
dictionary = json.loads(json_data)

In [ ]:
import itertools
keys = dictionary.keys()
target_img = []
query_img = []
for key in itertools.islice(keys, 2):
    value = dictionary[key]
    query_image = cv2.imread( os.path.join(image_folder, value['query']), cv2.IMREAD_GRAYSCALE)
    query_image = cv2.resize(query_image, (128, 128))
    query_img.append(query_image)

    print(value['similar'])
    target_img.append(value['similar'])


['100001.jpg', '100002.jpg']
['100101.jpg']


In [ ]:
# Create SIFT object
sift = cv2.SIFT_create()

def key_point(query_image):
    # Detect keypoints and compute descriptors for the target image
    kp_target, des_target = sift.detectAndCompute(query_image, None)
    des_target = des_target.astype(np.float32)
    return kp_target, des_target


# Set up variables to store the maximum number of matches and the image with the maximum matches
max_matches = 0
max_matches_img = None


In [ ]:
top_matches = []
final_result = {}

In [ ]:
i = 0
for img in query_img:
    kp_target, des_target = key_point(img)    # Loop over all images in the folder
    for filename in os.listdir('./images'):
        # Load the image from file and convert it to grayscale
        img_path = os.path.join('./images', filename)
        img_data = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img_data = cv2.resize(img, (128, 128))

        # Detect keypoints and compute descriptors for the current image
        kp, des = sift.detectAndCompute(img_data, None)
        if des is None:
            continue

        des = des.astype(np.float32)

        # Compute matches using FLANN-based matcher
        matcher = cv2.FlannBasedMatcher()
        matches = matcher.match(des_target, des)
        num_matches = len(matches)

        print(num_matches)
        # Store the current image and its number of matches in the top_matches list
        top_matches.append((num_matches, img_path))

        # Keep only the top 4 matches in the list
        top_matches = heapq.nlargest(4, top_matches, key=lambda x: x[0])
        final_result[i] = top_matches
        i +=1
        print(top_matches)
        print(final_result)

        

        
        # # Check if the current image has the maximum number of matches
        # if num_matches > max_matches:
        #     max_matches = num_matches
        #     max_matches_img = img_path
